In [ ]:
import pandas as pd

In [ ]:
import warnings, logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical

'''from preprocess import *'''

def load_transfer_model():
    # ImageNet으로 훈련된 ResNet-50 모델을 불러옵니다.
    # 가장 마지막의 classification layer는 포함하지 않습니다.

    base_model = ResNet50(input_shape=(84, 84, 3), include_top=False, weights='imagenet') # input_shape를 바꿔야 함, 예상모양이 발견모양과 호환되지 않는다.
    
    # TODO: [지시사항 1번] 모델이 학습되지 않도록 설정하세요.
    base_model.trainable = False
    
    # TODO: [지시사항 2번] 지시사항에 따라 layer를 추가하세요. ## UpSampling2D : 데이터의 행과 열을 각각 size[0]과 size[1]만큼 반복
    # 아는 것 : transfer_model, sequential, upsampling2d, interpolation, dense, activation='softmax',base_model  , 모르는 것 : globalaveragepooling2d, size
    transfer_model = Sequential([
        layers.UpSampling2D(size=(3, 3), interpolation='bilinear'),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(10, activation='softmax'),
    ])
  
  #레이어 "up_sampling2d_5"의 입력 0이 레이어와 호환되지 않습니다: 예상 ndim=4, 발견 ndim=3. 전체 모양 수신: (500, 28, 28)
  #수신된 호출 인수:
     #• 입력=tf.Tensor(모양=(500, 28, 28), dtype=uint8)
      #• 교육=참
      #• 마스크=없음


    return transfer_model
 # interpolation : 알려진 값 사이(중간)에 위치한 값을 추정
 # bilinear interpolation : 쌍선형 보간법(1차원에서의 선형보간법을 2차원으로 확장)

'''class fashion_mnist:
  def load_data():
    return (x_train, y_train), (x_test, y_test)'''
def main(transfer_model=None, epochs=3):
    np.random.seed(81)
    
    #num_classes = 10
#fashion_mnist : 인스턴스, load_data() : 함수의 별명 클래스 안에 미리 해놓은 함수는 메소드라고 부름
    dataloader = keras.datasets.fashion_mnist

    (x_train, y_train), (x_test, y_test) = dataloader.load_data() #ValueError: 압축을 풀기에 값이 충분하지 않습니다(예상 4개, 2개 있음)

    # fashion_mnist를 왜 불러오고 싶었는지 : fashion_mnist가 눈에 더 잘 띄어서, 데이터셋 이름이라서
    x_train, y_train, x_test, y_test = x_train[:5000], y_train[:5000], x_test[:100], y_test[:100]
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_train2 = x_train.copy()
    x_train3 = x_train.copy()
    print(x_train2.shape)
    one_hot = to_categorical(y_train)
    print(one_hot)
    #print(x)
    #print("x_train2 :" , x_train2)
    x = np.block([x_train, x_train2, x_train3])
    print(x.shape)
    #print(x_train.shape)
    #레이어 "resnet50"의 입력 0은 레이어와 호환되지 않습니다: 예상 모양=(없음, 96, 96, 3), 발견된 모양=(500, 84, 84, 1)
    # 아는 것 : , 모르는 것 : 예상모양. 발견된 모양, resnet50
    
    if transfer_model is None:
        transfer_model = load_transfer_model()
    
    # [지시사항 3번] 모델 학습을 위한 Optimizer, loss 함수, 평가 지표를 설정하세요.
    optimizer = Adam(learning_rate=0.001)
    transfer_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    #print(y_train.shape)
    # 모델을 학습시킵니다.
    hist = transfer_model.fit(x, one_hot, epochs=epochs, batch_size=500) # 아는 것 : hist, x_train, y_train, epochs, batch_size # 모르는 것 : transfer_model.fit, # fit(x,y)
    
    # 완성된 모델을 확인해봅니다.
    print()
    transfer_model.summary()
    
    # 테스트 데이터로 모델 성능을 확인합니다.
    loss, accuracy = transfer_model.evaluate(x_test, y_test)
    
    print('\n훈련된 모델의 테스트 정확도는 {:.3f}% 입니다.\n'.format(accuracy * 100))
    
    return optimizer, hist

main()

(5000, 28, 28, 1)
[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(5000, 28, 28, 3)
94781440/94765736 [==============================] - 1s 0us/step
Epoch 1/3
 2/10 [=====>........................] - ETA: 1:45 - loss: 3.4343 - accuracy: 0.1210